In [1]:
import numpy as np
import tensorflow as tf
import random

class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self.build_network()

    
    # h_size: hidden size, l_rate: learning rate
    def build_network(self, h_size = 10, l_rate = 1e-1):
        with tf.variable_scope(self.net_name):
            self._X=tf.placeholder(dtype=tf.float32, shape=[None, self.input_size])

            # First layer of weights
            W1 = tf.get_variable('W1', shape=[self.input_size, h_size], 
                                 initializer=tf.contrib.layers.xavier_initializer())
            layer1 = tf.nn.tanh(tf.matmul(self._X, W1))

            # Second layer of weights
            W2 = tf.get_variable('W2', shape=[h_size, self.output_size], 
                                 initializer=tf.contrib.layers.xavier_initializer())

            # Q prediction
            self._Qpred = tf.matmul(layer1, W2)
        
        # We need to define the parts of the network needed for learning a polilcy
        self._Y = tf.placeholder(dtype=tf.float32, shape=[None, self.output_size])
        
        # Loss function
        self._loss = tf.reduce_sum(tf.square(self._Y - self._Qpred))
        
        # Learning
        self._train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self._loss)
    

    def predict(self, state):
        s_t = np.reshape(state, [1,self.input_size])
        return self.session.run(self._Qpred, feed_dict={self._X: s_t})

    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})

In [2]:
from collections import deque
import gym

env = gym.make("CartPole-v0")
env._max_episode_steps = 50000

# Constant defining our neural network
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

dis = 0.9
REPLAY_MEMORY = 50000

def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, input_size)
    y_stack = np.empty(0).reshape(0, output_size)
    
    # Get sotred information from the buffer
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)

        # terminal?
        if done:
            Q[0, action] = reward
        else:
            # get target from target DQN(Q')
            Q[0, action] = reward + dis * np.max(targetDQN.predict(next_state))

        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
        
    # Train our network using target and predicted Q values on each episode
    return mainDQN.update(x_stack, y_stack)

def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    # Copy variables src_scope to dest_scope
    op_holder = []
    
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value()))
        
    return op_holder

def bot_play(mainDQN):
    # See our trained network in action
    s = env.reset()
    reward_sum = 0
    while True:
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print("Total score: {}".format(reward_sum))
            break
            
def main():
    max_episodes = 5000
    # store the previous observations in replay memory
    replay_buffer = deque()
    
    with tf.Session() as sess:
        mainDQN = DQN(sess, input_size, output_size, name="main")
        targetDQN = DQN(sess, input_size, output_size, name="target")
        tf.global_variables_initializer().run()
        
        # initial copy q_net -> target_net
        copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name="main")
        
        sess.run(copy_ops)
        
        for episode in range(max_episodes):
            e = 1. / ((episode / 10) + 1)
            done = False
            step_count = 0
            state = env.reset()
            
            while not done:
                if np.random.rand(1) < e:
                    action = env.action_space.sample()
                else:
                    # Choose an action by greedily from the Q-network
                    action = np.argmax(mainDQN.predict(state))
                    
                # Get new state and reward from environment
                next_state, reward, done, _ = env.step(action)
                if done: # Penalty
                    reward = -100
                
                # save the experience to our buffer
                replay_buffer.append((state, action, reward, next_state, done))
                if len(replay_buffer) > REPLAY_MEMORY:
                    replay_buffer.popleft()
                
                state = next_state
                step_count += 1
                if step_count > 10000: # Good enough, Let's move on
                    break
                    
            print("Episode: {} steps: {}".format(episode, step_count))
            if step_count > 10000:
                pass
                break
            
            if episode % 10 == 1: # train every 10 episodes
                # Get a random batch of experiences.
                for _ in range(50):
                    #Minibatch works better
                    minibatch = random.sample(replay_buffer, 10)
                    loss, _ = replay_train(mainDQN, targetDQN, minibatch)
                print("Loss :", loss)
                
                #Copy q_net -> target_net
                sess.run(copy_ops)
            
        bot_play(mainDQN)

[2017-08-04 13:05:25,071] Making new env: CartPole-v0


In [ ]:
if __name__ == "__main__":
    main()

Episode: 0 steps: 12
Episode: 1 steps: 15
Loss : 525.166
Episode: 2 steps: 13
Episode: 3 steps: 20
Episode: 4 steps: 19
Episode: 5 steps: 13
Episode: 6 steps: 16
Episode: 7 steps: 9
Episode: 8 steps: 12
Episode: 9 steps: 10
Episode: 10 steps: 9
Episode: 11 steps: 16
Loss : 437.87
Episode: 12 steps: 10
Episode: 13 steps: 13
Episode: 14 steps: 10
Episode: 15 steps: 8
Episode: 16 steps: 11
Episode: 17 steps: 10
Episode: 18 steps: 11
Episode: 19 steps: 9
Episode: 20 steps: 10
Episode: 21 steps: 8
Loss : 11593.4
Episode: 22 steps: 8
Episode: 23 steps: 15
Episode: 24 steps: 10
Episode: 25 steps: 10
Episode: 26 steps: 9
Episode: 27 steps: 9
Episode: 28 steps: 9
Episode: 29 steps: 8
Episode: 30 steps: 10
Episode: 31 steps: 11
Loss : 311.748
Episode: 32 steps: 43
Episode: 33 steps: 21
Episode: 34 steps: 22
Episode: 35 steps: 25
Episode: 36 steps: 24
Episode: 37 steps: 31
Episode: 38 steps: 33
Episode: 39 steps: 29
Episode: 40 steps: 19
Episode: 41 steps: 28
Loss : 214.138
Episode: 42 steps: 10


Loss : 154.707
Episode: 342 steps: 9
Episode: 343 steps: 16
Episode: 344 steps: 14
Episode: 345 steps: 10
Episode: 346 steps: 10
Episode: 347 steps: 11
Episode: 348 steps: 9
Episode: 349 steps: 15
Episode: 350 steps: 9
Episode: 351 steps: 9
Loss : 126.716
Episode: 352 steps: 22
Episode: 353 steps: 23
Episode: 354 steps: 21
Episode: 355 steps: 18
Episode: 356 steps: 22
Episode: 357 steps: 19
Episode: 358 steps: 28
Episode: 359 steps: 22
Episode: 360 steps: 36
Episode: 361 steps: 39
Loss : 97.0168
Episode: 362 steps: 30
Episode: 363 steps: 37
Episode: 364 steps: 37
Episode: 365 steps: 61
Episode: 366 steps: 44
Episode: 367 steps: 91
Episode: 368 steps: 35
Episode: 369 steps: 34
Episode: 370 steps: 40
Episode: 371 steps: 75
Loss : 55.7952
Episode: 372 steps: 9
Episode: 373 steps: 9
Episode: 374 steps: 9
Episode: 375 steps: 9
Episode: 376 steps: 10
Episode: 377 steps: 8
Episode: 378 steps: 9
Episode: 379 steps: 9
Episode: 380 steps: 8
Episode: 381 steps: 9
Loss : 8360.56
Episode: 382 steps